In [19]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

from sklearn.metrics import mean_squared_error
from sklearn import linear_model
from sklearn.model_selection import KFold
pd.options.display.max_columns = 999

In [2]:
data = pd.read_table('AmesHousing.tsv')

In [3]:
def transform_features(DF):
    return DF

def select_features(df):
    return df[["Gr Liv Area", "SalePrice"]]


def train_and_test(df):
    train = df.iloc[:1460]
    test = df.iloc[1460:]
    train = transform_features(train)
    numerical_train = train.select_dtypes(include=['integer','float'])
    numerical_test = test.select_dtypes(include=['integer','float'])
    features = numerical_train.columns.drop('SalePrice')
    
    lr = LinearRegression()
    lr.fit(train[features],train['SalePrice'])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(y_pred = predictions, y_true = test['SalePrice'])
    rmse = np.sqrt(mse)
    return rmse
transf = transform_features(data)
feature_filter = select_features(transf)
rmse = train_and_test(feature_filter)

rmse

57088.25161263909

In [4]:
temp = data.copy()

##Feature Engineering
Handle missing values:
All columns:

Drop any with 5% or more missing values for now.

Text columns:
Drop any with 1 or more missing values for now.

Numerical columns:
For columns with missing values, fill in with the most common value in that column


In [5]:
missing_val = temp.isnull().sum()

drop_cols = [x for x in missing_val.index if missing_val[x] / len(temp) > .05] 
temp.drop(columns = drop_cols, inplace=True)

text_columns = temp.select_dtypes(include='object')
text_missing = text_columns.isnull().sum()
drop_text_cols = [x for x in text_missing.index if text_missing[x] > 0]
temp.drop(columns=drop_text_cols, inplace=True)

temp.columns

##Filling in numeric columns with the most common value

fixable_cols = temp.select_dtypes(include=['float', 'int']).isnull().sum()
fixable_cols = fixable_cols[fixable_cols > 0]
fixable_cols

modes = {}
for col in fixable_cols.index:
    mode = temp[col].mode()[0]
    modes[col] = mode

modes
temp = temp.fillna(modes)

##Creating new variable called years until remodeled to reflect how long it had been since the house had been remodeled.

temp['years_until_remod'] = temp['Year Built'] - temp['Year Remod/Add']

years_sold = temp['Yr Sold'] - temp['Year Built']


sold_after_remod = temp['Yr Sold'] - temp['Year Remod/Add']
print(sold_after_remod[sold_after_remod <0])

temp = temp.drop(axis = 1, index = [2180,1702,2180,2181])

#Dropping columns that aren't useful for machine learning

temp = temp.drop(columns=['Order','PID'])

#Dropping columns that might leak data about the sale price.

temp.drop(columns=['Yr Sold','Mo Sold','Sale Condition','Sale Type'], inplace=True)

print(temp.shape)

1702   -1
2180   -2
2181   -1
dtype: int64
(2927, 59)


In [6]:
def transform_features(temp):
    
    missing_val = temp.isnull().sum()

    drop_cols = [x for x in missing_val.index if missing_val[x] / len(temp) > .05] 
    temp.drop(columns = drop_cols, inplace=True)

    text_columns = temp.select_dtypes(include='object')
    text_missing = text_columns.isnull().sum()
    drop_text_cols = [x for x in text_missing.index if text_missing[x] > 0]
    temp.drop(columns=drop_text_cols, inplace=True)

    temp.columns

    ##Filling in numeric columns with the most common value

    fixable_cols = temp.select_dtypes(include=['float', 'int']).isnull().sum()
    fixable_cols = fixable_cols[fixable_cols > 0]
    fixable_cols

    modes = {}
    for col in fixable_cols.index:
        mode = temp[col].mode()[0]
        modes[col] = mode

    modes
    temp = temp.fillna(modes)

    ##Creating new variable called years until remodeled to reflect how long it had been since the house had been remodeled.

    temp['years_until_remod'] = temp['Year Built'] - temp['Year Remod/Add']

    years_sold = temp['Yr Sold'] - temp['Year Built']


    sold_after_remod = temp['Yr Sold'] - temp['Year Remod/Add']

    temp = temp.drop(axis = 1, index = [2180,1702,2180,2181])

    #Dropping columns that aren't useful for machine learning

    temp = temp.drop(columns=['Order','PID'])

    #Dropping columns that might leak data about the sale price.

    temp.drop(columns=['Yr Sold','Mo Sold','Sale Condition','Sale Type', 'Year Remod/Add','Year Built'], inplace=True)

    return temp

def select_features(df):
    return df[["Gr Liv Area", "SalePrice"]]
    

def train_and_test(df):
    train = df.iloc[:1460]
    test = df.iloc[1460:]
    numerical_train = train.select_dtypes(include=['integer','float'])
    numerical_test = test.select_dtypes(include=['integer','float'])
    features = numerical_train.columns.drop('SalePrice')
    
    lr = LinearRegression()
    lr.fit(train[features],train['SalePrice'])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(y_pred = predictions, y_true = test['SalePrice'])
    rmse = np.sqrt(mse)
    return rmse

In [7]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

55275.36731241307

In [16]:

##Drop columns with low correlation
data = pd.read_table('AmesHousing.tsv')

data = transform_features(data)

numerical_features = data.select_dtypes(include=['int','float'])

numer_corrs = numerical_features.corr()['SalePrice']
numer_corrs = numer_corrs[numer_corrs > .4].sort_values()

numer_corrs

drop_corrs = numerical_features.corr()['SalePrice']

data = data.drop(columns = drop_corrs[drop_corrs < .4].index)

numer_corrs

## Moving onto Columns that are meant to be categorical. 

    #Numerical Categorical
Fireplaces
TotRms AbvGrd
Full Bath
Garage Cars
Overall Qual
    ## Object Categorical/Nominal

['MS Zoning','Street','Land Contour','Lot Config','Neighborhood','Condition1','Condition2','Bldg Type',
'Bldg Type','House Style','Roof Style','Roof Matl']


data.select_dtypes(include=['object']).columns

nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]

len(nominal_features)

transformed_cols = []
for col in nominal_features:
    if col in data.columns:
        transformed_cols.append(col)

print(transformed_cols, len(transformed_cols))

trans_counts = data[transformed_cols].apply(lambda col: len(col.value_counts()))

trans_counts_drop = trans_counts[trans_counts <10]

data.drop(columns=trans_counts_drop.index,axis =0,inplace= True)


text_columns = data.select_dtypes(include='object')

for each in text_columns:
    data[each] = data[each].astype('category')

data = pd.concat([
    data, 
    pd.get_dummies(data.select_dtypes(include=['category']))
], axis=1)

In [6]:
def transform_features(temp):
    
    missing_val = temp.isnull().sum()

    drop_cols = [x for x in missing_val.index if missing_val[x] / len(temp) > .05] 
    temp.drop(columns = drop_cols, inplace=True)

    text_columns = temp.select_dtypes(include='object')
    text_missing = text_columns.isnull().sum()
    drop_text_cols = [x for x in text_missing.index if text_missing[x] > 0]
    temp.drop(columns=drop_text_cols, inplace=True)

    temp.columns

    ##Filling in numeric columns with the most common value

    fixable_cols = temp.select_dtypes(include=['float', 'int']).isnull().sum()
    fixable_cols = fixable_cols[fixable_cols > 0]
    fixable_cols

    modes = {}
    for col in fixable_cols.index:
        mode = temp[col].mode()[0]
        modes[col] = mode

    modes
    temp = temp.fillna(modes)

    ##Creating new variable called years until remodeled to reflect how long it had been since the house had been remodeled.

    temp['years_until_remod'] = temp['Year Built'] - temp['Year Remod/Add']

    years_sold = temp['Yr Sold'] - temp['Year Built']


    sold_after_remod = temp['Yr Sold'] - temp['Year Remod/Add']

    temp = temp.drop(axis = 1, index = [2180,1702,2180,2181])

    #Dropping columns that aren't useful for machine learning

    temp = temp.drop(columns=['Order','PID'])

    #Dropping columns that might leak data about the sale price.

    temp.drop(columns=['Yr Sold','Mo Sold','Sale Condition','Sale Type', 'Year Remod/Add','Year Built'], inplace=True)

    return temp

def select_features(df, coeff_threshold=0.4, uniq_threshold=10):
    numerical_df = df.select_dtypes(include=['int', 'float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1)
    
    return df
    

def train_and_test(df):
    train = df.iloc[:1460]
    test = df.iloc[1460:]
    numerical_train = train.select_dtypes(include=['integer','float'])
    numerical_test = test.select_dtypes(include=['integer','float'])
    features = numerical_train.columns.drop('SalePrice')
    
    lr = LinearRegression()
    lr.fit(train[features],train['SalePrice'])
    predictions = lr.predict(test[features])
    mse = mean_squared_error(y_pred = predictions, y_true = test['SalePrice'])
    rmse = np.sqrt(mse)
    return rmse

In [7]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df)

rmse

33447.420154009684

In [10]:
def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()
    
    if k == 0:
        
        train = df.iloc[:1460]
        test = df.iloc[1460:]
        
        lr.fit(train[features],train['SalePrice'])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(y_pred = predictions, y_true = test['SalePrice'])
        rmse = np.sqrt(mse)
        return rmse
    if k ==1:
        shuffled_df = df.sample(frac=1, )
        fold_one = df.iloc[:1460]
        fold_two = df.iloc[1460:]

       
        lr.fit(fold_one[features],fold_one['SalePrice'])
        predictions_one = lr.predict(fold_two[features])
        one_mse = mean_squared_error(y_pred = predictions_one, y_true = fold_two['SalePrice'])
        one_rmse = np.sqrt(one_mse)
        
        lr.fit(fold_two[features], fold_two['SalePrice'])
        predictions_two = lr.predict(fold_one[features])
        two_mse = mean_squared_error(y_pred=predictions_two, y_true=fold_one['SalePrice'])
        two_rmse = np.sqrt(two_mse)
        
        print(one_rmse, two_rmse)
        avg_rmse = np.mean([one_rmse,two_rmse])
        return(avg_rmse)
    else:
        kf = KFold(n_splits = k, shuffle=True)
        rmse_values =[]
        for train_index, test_index in kf.fit(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features],train['SalePrice'])
            predictions_temp = lr.predict(test[features])
            mse = mean_squared_error(y_pred = predictions_temp, y_true = test['SalePrice'])
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return(avg_rmse)
            
    
    
    
    
    
    
    
    
    

In [13]:
def transform_features(temp):
    
    missing_val = temp.isnull().sum()

    drop_cols = [x for x in missing_val.index if missing_val[x] / len(temp) > .05] 
    temp.drop(columns = drop_cols, inplace=True)

    text_columns = temp.select_dtypes(include='object')
    text_missing = text_columns.isnull().sum()
    drop_text_cols = [x for x in text_missing.index if text_missing[x] > 0]
    temp.drop(columns=drop_text_cols, inplace=True)

    temp.columns

    ##Filling in numeric columns with the most common value

    fixable_cols = temp.select_dtypes(include=['float', 'int']).isnull().sum()
    fixable_cols = fixable_cols[fixable_cols > 0]
    fixable_cols

    modes = {}
    for col in fixable_cols.index:
        mode = temp[col].mode()[0]
        modes[col] = mode

    modes
    temp = temp.fillna(modes)

    ##Creating new variable called years until remodeled to reflect how long it had been since the house had been remodeled.

    temp['years_until_remod'] = temp['Year Built'] - temp['Year Remod/Add']

    years_sold = temp['Yr Sold'] - temp['Year Built']


    sold_after_remod = temp['Yr Sold'] - temp['Year Remod/Add']

    temp = temp.drop(axis = 1, index = [2180,1702,2180,2181])

    #Dropping columns that aren't useful for machine learning

    temp = temp.drop(columns=['Order','PID'])

    #Dropping columns that might leak data about the sale price.

    temp.drop(columns=['Yr Sold','Mo Sold','Sale Condition','Sale Type', 'Year Remod/Add','Year Built'], inplace=True)

    return temp

def select_features(df, coeff_threshold=0.4, uniq_threshold=10):
    numerical_df = df.select_dtypes(include=['int', 'float'])
    abs_corr_coeffs = numerical_df.corr()['SalePrice'].abs().sort_values()
    df = df.drop(abs_corr_coeffs[abs_corr_coeffs < coeff_threshold].index, axis=1)
    
    nominal_features = ["PID", "MS SubClass", "MS Zoning", "Street", "Alley", "Land Contour", "Lot Config", "Neighborhood", 
                    "Condition 1", "Condition 2", "Bldg Type", "House Style", "Roof Style", "Roof Matl", "Exterior 1st", 
                    "Exterior 2nd", "Mas Vnr Type", "Foundation", "Heating", "Central Air", "Garage Type", 
                    "Misc Feature", "Sale Type", "Sale Condition"]
    
    transform_cat_cols = []
    for col in nominal_features:
        if col in df.columns:
            transform_cat_cols.append(col)

    uniqueness_counts = df[transform_cat_cols].apply(lambda col: len(col.value_counts())).sort_values()
    drop_nonuniq_cols = uniqueness_counts[uniqueness_counts > 10].index
    df = df.drop(drop_nonuniq_cols, axis=1)
    
    text_cols = df.select_dtypes(include=['object'])
    for col in text_cols:
        df[col] = df[col].astype('category')
    df = pd.concat([df, pd.get_dummies(df.select_dtypes(include=['category']))], axis=1)
    
    return df

def train_and_test(df, k=0):
    numeric_df = df.select_dtypes(include=['integer', 'float'])
    features = numeric_df.columns.drop("SalePrice")
    lr = linear_model.LinearRegression()
    
    if k == 0:
        
        train = df.iloc[:1460]
        test = df.iloc[1460:]
        
        lr.fit(train[features],train['SalePrice'])
        predictions = lr.predict(test[features])
        mse = mean_squared_error(y_pred = predictions, y_true = test['SalePrice'])
        rmse = np.sqrt(mse)
        return rmse
    if k ==1:
        shuffled_df = df.sample(frac=1, )
        fold_one = df.iloc[:1460]
        fold_two = df.iloc[1460:]

       
        lr.fit(fold_one[features],fold_one['SalePrice'])
        predictions_one = lr.predict(fold_two[features])
        one_mse = mean_squared_error(y_pred = predictions_one, y_true = fold_two['SalePrice'])
        one_rmse = np.sqrt(one_mse)
        
        lr.fit(fold_two[features], fold_two['SalePrice'])
        predictions_two = lr.predict(fold_one[features])
        two_mse = mean_squared_error(y_pred=predictions_two, y_true=fold_one['SalePrice'])
        two_rmse = np.sqrt(two_mse)
        
        print(one_rmse, two_rmse)
        avg_rmse = np.mean([one_rmse,two_rmse])
        return(avg_rmse)
    else:
        kf = KFold(n_splits = k, shuffle=True)
        rmse_values =[]
        for train_index, test_index in kf.split(df):
            train = df.iloc[train_index]
            test = df.iloc[test_index]
            lr.fit(train[features],train['SalePrice'])
            predictions_temp = lr.predict(test[features])
            mse = mean_squared_error(y_pred = predictions_temp, y_true = test['SalePrice'])
            rmse = np.sqrt(mse)
            rmse_values.append(rmse)
        print(rmse_values)
        avg_rmse = np.mean(rmse_values)
        return(avg_rmse)


In [15]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=4)

rmse

[25492.00348398577, 29195.86503268019, 26257.78433398567, 36517.49390408304]


29365.786688683664

In [16]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)
rmse = train_and_test(filtered_df, k=4)

rmse

[28969.309872520593, 36078.1581345842, 28745.35414438099, 24240.59661584251]


29508.354691832075

In [17]:
df = pd.read_csv("AmesHousing.tsv", delimiter="\t")
transform_df = transform_features(df)
filtered_df = select_features(transform_df)

In [31]:
transform_df.select_dtypes(include=['int']).columns

Index(['MS SubClass', 'Lot Area', 'Overall Qual', 'Overall Cond', '1st Flr SF',
       '2nd Flr SF', 'Low Qual Fin SF', 'Gr Liv Area', 'Full Bath',
       'Half Bath', 'Bedroom AbvGr', 'Kitchen AbvGr', 'TotRms AbvGrd',
       'Fireplaces', 'Wood Deck SF', 'Open Porch SF', 'Enclosed Porch',
       '3Ssn Porch', 'Screen Porch', 'Pool Area', 'Misc Val', 'SalePrice',
       'years_until_remod'],
      dtype='object')

In [32]:
filtered_df.select_dtypes(include=['category']).columns

Index(['MS Zoning', 'Street', 'Lot Shape', 'Land Contour', 'Utilities',
       'Lot Config', 'Land Slope', 'Condition 1', 'Condition 2', 'Bldg Type',
       'House Style', 'Roof Style', 'Roof Matl', 'Exter Qual', 'Exter Cond',
       'Foundation', 'Heating', 'Heating QC', 'Central Air', 'Kitchen Qual',
       'Functional', 'Paved Drive'],
      dtype='object')

In [42]:
cols =[]
for i in transform_df.select_dtypes(include=['int']).columns:
    if i in filtered_df.columns:
        cols.append(i)

In [43]:
cols

['Overall Qual',
 '1st Flr SF',
 'Gr Liv Area',
 'Full Bath',
 'TotRms AbvGrd',
 'Fireplaces',
 'SalePrice']

dtype('int64')